In [ ]:
#@title Installazione librerie
!pip install transformers diffusers torchvision

In [ ]:
#@title Upload foto riferimenti

from google.colab import files
import os

def save_uploaded_files(save_directory):
    uploaded = files.upload()  # apre il selettore per l'upload

    os.makedirs(save_directory, exist_ok=True)

    for filename, content in uploaded.items():
        filepath = os.path.join(save_directory, filename)
        with open(filepath, 'wb') as f:
            f.write(content)
        print(f"✅ Salvato: {filepath}")

In [ ]:
#@markdown Inserisci il percorso in cui salvare i file (verrà creato se non esiste):

save_path = "./dataset/andrea/Dreambooth2" #@param {type:"string"}
save_uploaded_files(save_path)

In [ ]:
#@title Upload file rar ed estrazione

!pip install rarfile unrar

import rarfile

def upload_and_extract_rar(save_directory):
    uploaded = files.upload()  # carica file RAR

    os.makedirs(save_directory, exist_ok=True)

    for filename in uploaded.keys():
        rar_path = os.path.join(save_directory, filename)
        with open(rar_path, 'wb') as f:
            f.write(uploaded[filename])

        print(f"✅ RAR salvato: {rar_path}")

        # estrai con rarfile
        with rarfile.RarFile(rar_path) as rf:
            rf.extractall(save_directory)
            print(f"📂 Estratto in: {save_directory}")

        os.remove(rar_path)


In [ ]:
#@markdown Inserisci il percorso in cui salvare i file (verrà creato se non esiste):

save_path = "./dataset/test/" #@param {type:"string"}
upload_and_extract_rar(save_path)

In [ ]:
#@title Funzione per computare il CLIP-I di un img su un set di riferimenti e calcolarne il valore medio

import os
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

#Calcola la CLIP similarity tra un'immagine generata e un set di immagini di riferimento.
def compute_clip_similarity(ref_dir, gen_image_path, device="cpu"):

    # Carica il modello CLIP
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Trova immagini di riferimento
    valid_exts = [".jpg", ".jpeg", ".png"]
    ref_image_paths = [
        os.path.join(ref_dir, fname)
        for fname in os.listdir(ref_dir)
        if os.path.splitext(fname)[1].lower() in valid_exts
    ]

    if len(ref_image_paths) == 0:
        raise ValueError(f"Nessuna immagine valida trovata in {ref_dir}")

    # Carica immagini di riferimento
    ref_images = [Image.open(path).convert("RGB") for path in ref_image_paths]

    # Prepara immagini di riferimento
    inputs_ref = clip_processor(images=ref_images, return_tensors="pt", padding=True).to(device)

    # Estrai embeddings immagini di riferimento
    with torch.no_grad():
        ref_features = clip_model.get_image_features(**inputs_ref)

    # Normalizza embeddings
    ref_features = ref_features / ref_features.norm(p=2, dim=-1, keepdim=True)

    # Carica immagine generata
    image_gen = Image.open(gen_image_path).convert("RGB")

    # Prepara immagine generata
    inputs_gen = clip_processor(images=[image_gen], return_tensors="pt").to(device)

    # Estrai embedding immagine generata
    with torch.no_grad():
        gen_feature = clip_model.get_image_features(**inputs_gen)

    # Normalizza embedding immagine generata
    gen_feature = gen_feature / gen_feature.norm(p=2, dim=-1, keepdim=True)

    # Calcola cosine similarity per ogni immagine di riferimento
    similarities = []
    for ref_feat in ref_features:
        sim = torch.nn.functional.cosine_similarity(gen_feature[0], ref_feat, dim=0)
        similarities.append(sim.item())

    # Calcola media delle similarità
    mean_similarity = sum(similarities) / len(similarities)

    # Output
    result = {
        "ref_image_paths": ref_image_paths,
        "similarities": similarities,
        "mean_similarity": mean_similarity
    }

    return result

In [ ]:
#@title Computazione inserendo i percorsi che ci servono

result = compute_clip_similarity("/content/dataset/cami/Mudi", "/content/dataset/test/testmetriche/sdxl/andrea_cami1.png")

print(f"Similarità per ogni immagine di riferimento:")
for path, sim in zip(result["ref_image_paths"], result["similarities"]):
    print(f"{path}: {sim:.4f}")

print(f"Media CLIP similarity: {result['mean_similarity']:.4f}")


Similarità per ogni immagine di riferimento:
/content/dataset/cami/Mudi/b00.jpg: 0.6011
/content/dataset/cami/Mudi/b03.jpg: 0.5657
/content/dataset/cami/Mudi/b02.jpg: 0.5519
/content/dataset/cami/Mudi/b04.jpg: 0.5899
/content/dataset/cami/Mudi/b01.jpg: 0.5344
Media CLIP similarity: 0.5686
